# How to use AutoGluon for Kaggle competitions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/advanced/tabular-kaggle.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/advanced/tabular-kaggle.ipynb)



This tutorial will teach you how to use AutoGluon to become a serious Kaggle competitor without writing lots of code.
We first outline the general steps to use AutoGluon in Kaggle contests. Here, we assume the competition involves tabular data which are stored in one (or more) CSV files.

1) Run Bash command: pip install kaggle

2) Navigate to: https://www.kaggle.com/account and create an account (if necessary).
Then , click on "Create New API Token" and move downloaded file to this location on your machine: `~/.kaggle/kaggle.json`. For troubleshooting, see [Kaggle API instructions](https://www.kaggle.com/docs/api).

3) To download data programmatically: Execute this Bash command in your terminal:

`kaggle competitions download -c [COMPETITION]`

Here, [COMPETITION] should be replaced by the name of the competition you wish to enter.
Alternatively, you can download data manually: Just navigate to website of the Kaggle competition you wish to enter, click "Download All", and accept the competition's terms.

4) If the competition's training data is comprised of multiple CSV files, use [pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) to properly merge/join them into a single data table where rows = training examples, columns = features.

5) Run autogluon `fit()` on the resulting data table.

6) Load the test dataset from competition (again making the necessary merges/joins to ensure it is in the exact same format as the training data table), and then call autogluon `predict()`.  Subsequently use [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) to load the competition's `sample_submission.csv` file into a DataFrame, put the AutoGluon predictions in the right column of this DataFrame, and finally save it as a CSV file via [pandas.to_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html). If the competition does not offer a sample submission file, you will need to create the submission file yourself by appropriately reformatting AutoGluon's test predictions.

7) Submit your predictions via Bash command:

`kaggle competitions submit -c [COMPETITION] -f [FILE] -m ["MESSAGE"]`

Here, [COMPETITION] again is the competition's name, [FILE] is the name of the CSV file you created with your predictions, and ["MESSAGE"] is a string message you want to record with this submitted entry. Alternatively, you can  manually upload your file of predictions on the competition website.

8) Finally, navigate to competition leaderboard website to see how well your submission performed!
It may take time for your submission to appear.



Below, we demonstrate how to do steps (4)-(6) in Python for a specific Kaggle competition: [ieee-fraud-detection](https://www.kaggle.com/c/ieee-fraud-detection/).
This means you'll need to run the above steps with `[COMPETITION]` replaced by `ieee-fraud-detection` in each command.  Here, we assume you've already completed steps (1)-(3) and the data CSV files are available on your computer. To begin step (4), we first load the competition's training data into Python:

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shilpayr","key":"37d76a6f69d72827aef90afc88a3a117"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c ieee-fraud-detection


 82% 97.0M/118M [00:00<00:00, 1.01GB/s]
100% 118M/118M [00:00<00:00, 880MB/s]  


In [ ]:
!unzip ieee-fraud-detection.zip -d data/


Archive:  ieee-fraud-detection.zip
  inflating: data/sample_submission.csv  
  inflating: data/test_identity.csv  
  inflating: data/test_transaction.csv  
  inflating: data/train_identity.csv  
  inflating: data/train_transaction.csv  


In [ ]:
!pip -q install -U autogluon kaggle

import os, pandas as pd, numpy as np
from pathlib import Path

# Make a workspace folder
WORKDIR = Path("/content/ag_workspace")
WORKDIR.mkdir(parents=True, exist_ok=True)

print("All set. If you see no errors, move to the next cell!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.9/454.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.4 MB/s eta 0:00:

In [ ]:

import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = 'data/'  # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'AutoGluonModels/'  # where to store trained models

train_identity = pd.read_csv(directory+'train_identity.csv')
train_transaction = pd.read_csv(directory+'train_transaction.csv')


Since the training data for this competition is comprised of multiple CSV files, we just first join them into a single large table (with rows = examples, columns = features) before applying AutoGluon:

In [ ]:

train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')



Note that a left-join on the `TransactionID` key happened to be most appropriate for this Kaggle competition, but for others involving multiple training data files, you will likely need to use a different join strategy (always consider this very carefully). Now that all our training data resides within a single table, we can apply AutoGluon. Below, we specify the `presets` argument to maximize AutoGluon's predictive accuracy which usually requires that you run `fit()` with longer time limits (3600s below should likely be increased in your run):


'''predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='best_quality', time_limit=3600
)

results = predictor.fit_summary()'''



In [ ]:
from autogluon.tabular import TabularPredictor

# (1) OPTIONAL: if your dataset is huge, use a small sample for your demo run
# Comment this out later if you want to train on full data.
demo_frac = 0.25  # try 0.25 (25%). Increase if it runs fine.
train_demo = train_data.sample(frac=demo_frac, random_state=42)

# (2) Fit with memory-friendly settings
predictor = TabularPredictor(
    label=label,
    eval_metric=eval_metric,
    path=save_path,
    verbosity=3
).fit(
    train_data=train_demo,             # use train_data for full run; train_demo for a lighter demo
    presets='good_quality',            # lighter than 'best_quality'
    time_limit=1200,                   # lower total time; raise if you have headroom
    num_bag_folds=0,                   # 🔴 disables bagging (frees lots of RAM)
    num_stack_levels=0,                # 🔴 disables stacking/dynamic_stacking
    excluded_model_types=[             # drop memory-hungry models
        'NN_TORCH', 'CAT', 'KNN', 'XT'
    ],
    ag_args_fit={
        'num_cpus': 2,                 # reduce parallelism -> lower peak RAM
        'num_gpus': 0
    }
)

results = predictor.fit_summary()


Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
GPU Count:          0
Memory Avail:       3.12 GB / 12.67 GB (24.6%)
Disk Space Avail:   58.84 GB / 107.72 GB (54.6%)
Presets specified: ['good_quality']
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_cpus': 2, 'num_gpus': 0},
 'auto_stack': True,
 'excluded_model_types': ['NN_TORCH', 'CAT', 'KNN', 'XT'],
 'num_bag_folds': 0,
 'num_bag_sets': 1,
 'num_stack_levels': 0,
 'refit_full': True,
 'save_bag_folds': False,
 'set_best_to_refit_full': True}
Full kwargs:
{'_experimental_dynamic_hyperparameters': False,
 '_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_cpus': 2, 'num_gpus': 0},
 'auto_stack': True,
 '

Now, we use the trained AutoGluon Predictor to make predictions on the competition's test data. It is imperative that multiple test data files are joined together in the exact same manner as the training data. Because this competition is evaluated based on the AUC (Area under the ROC curve) metric, we ask AutoGluon for predicted class-probabilities rather than class predictions. In general, when to use `predict` vs `predict_proba` will depend on the particular competition.

In [ ]:
# 1) Imports
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

# ==== CONFIG: set paths ====
DATA_DIR = '/content/data/'             # where your CSVs are
MODEL_PATH = '/content/data/AutoGluonModels/'  # where your trained model was saved

# 2) Load trained predictor
predictor = TabularPredictor.load(MODEL_PATH)

# 3) Load test CSVs
test_identity = pd.read_csv(DATA_DIR + 'test_identity.csv')
test_transaction = pd.read_csv(DATA_DIR + 'test_transaction.csv')

# Quick sanity checks (prints help you confirm columns exist)
print('test_identity cols (first 10):', list(test_identity.columns[:10]))
print('test_transaction cols (first 10):', list(test_transaction.columns[:10]))

# 4) Merge EXACTLY like training
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

print('After merge, test_data shape:', test_data.shape)

# 5) Align test columns to what the model expects
#    These are the raw feature names used during training (label excluded)
train_features = predictor.features()
print('Model expects', len(train_features), 'features.')

# Add any missing expected cols (fill with NaN)
missing = [c for c in train_features if c not in test_data.columns]
if missing:
    print(f'Adding {len(missing)} missing columns with NaN:', missing[:8], '...' if len(missing) > 8 else '')
    for c in missing:
        test_data[c] = np.nan

# Drop any extra cols not seen in training
extra = [c for c in test_data.columns if c not in train_features]
if extra:
    print(f'Dropping {len(extra)} extra columns not used in training:', extra[:8], '...' if len(extra) > 8 else '')
    test_data = test_data.drop(columns=extra)

# Reorder columns to match training exactly
test_data = test_data[train_features]

# 6) Predict probabilities safely
probas = predictor.predict_proba(test_data)

# If binary classification: predict_proba returns a Series of P(positive class)
# If multiclass: it's a DataFrame; we pick positive class (1) or last column fallback
if isinstance(probas, pd.DataFrame):
    pos_class = 1 if 1 in probas.columns else probas.columns[-1]
    y_predproba = probas[pos_class].astype(float)
else:
    y_predproba = probas.astype(float)

print('First 5 probabilities:\n', y_predproba.head())

# 7) (Optional) Build a Kaggle submission for IEEE (TransactionID + isFraud)
submission = pd.DataFrame({
    'TransactionID': test_transaction['TransactionID'],
    'isFraud': y_predproba.values
})
submission_path = 'submission.csv'
submission.to_csv(submission_path, index=False)
print(f'Saved Kaggle submission to {submission_path} ✅')




test_identity cols (first 10): ['TransactionID', 'id-01', 'id-02', 'id-03', 'id-04', 'id-05', 'id-06', 'id-07', 'id-08', 'id-09']
test_transaction cols (first 10): ['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6']
After merge, test_data shape: (506691, 433)
Model expects 428 features.
Adding 38 missing columns with NaN: ['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08'] ...
Dropping 43 extra columns not used in training: ['V28', 'V117', 'V119', 'V147', 'V241', 'id-01', 'id-02', 'id-03'] ...
First 5 probabilities:
 0    0.010326
1    0.011221
2    0.026586
3    0.027764
4    0.018661
Name: 1, dtype: float64
Saved Kaggle submission to submission.csv ✅


When submitting predicted probabilities for classification competitions, it is imperative these correspond to the same class expected by Kaggle. For binary classification tasks, you can see which class AutoGluon's predicted probabilities correspond to via:

In [ ]:
predictor.positive_class

1

For multiclass classification tasks, you can see which classes AutoGluon's predicted probabilities correspond to via:

In [ ]:

predictor.class_labels  # classes in this list correspond to columns of predict_proba() output



[0, 1]

Now, let's get prediction probabilities for the entire test data, while only getting the positive class predictions by specifying:

In [ ]:

y_predproba = predictor.predict_proba(test_data, as_multiclass=False)



Now that we have made a prediction for each row in the test dataset, we can submit these predictions to Kaggle. Most Kaggle competitions provide a sample submission file, in which you can simply overwrite the sample predictions with your own as we do below:

In [ ]:

directory = '/content/data/'
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)



We have now completed steps (4)-(6) from the top of this tutorial. To submit your predictions to Kaggle, you can run the following command in your terminal (from the appropriate directory):

`kaggle competitions submit -c ieee-fraud-detection -f sample_submission.csv -m "my first submission"`

You can now play with different `fit()` arguments and feature-engineering techniques to try and maximize the rank of your submissions in the Kaggle Leaderboard!


**Tips to maximize predictive performance:**

   - Be sure to specify the appropriate evaluation metric if one is specified on the competition website! If you are unsure which metric is best, then simply do not specify this argument when invoking `fit()`; AutoGluon should still produce high-quality models by automatically inferring which metric to use.

   - If the training examples are time-based and the competition test examples come from future data, we recommend you reserve the most recently-collected training examples as a separate validation dataset passed to `fit()`. Otherwise, you do not need to specify a validation set yourself and AutoGluon will automatically partition the competition training data into its own training/validation sets.

   - Beyond simply specifying `presets = 'best_quality'`, you may play with more advanced `fit()` arguments such as: `num_bag_folds`, `num_stack_levels`, `num_bag_sets`, `hyperparameter_tune_kwargs`, `hyperparameters`, `refit_full`. However we recommend spending most of your time on feature-engineering and just specify `presets = 'best_quality'` inside the call to `fit()`.


**Troubleshooting:**

- Check that you have the right user-permissions on your computer to access the data files downloaded from Kaggle.

- For issues downloading Kaggle data or submitting predictions, check your Kaggle account setup and the [Kaggle FAQ](https://www.kaggle.com/general/14438).